In [9]:
import os
import numpy as np
import ChordDetection

In [10]:
class ChordPrediction:
    def __init__(self, audio_path):
        self.audio_path = audio_path
        self.DC = ChordDetection.DetectChords()
        self.DC.build_cnn_extractor()
        self.DC.build_crf()

    def predict(self):
        if os.path.exists(self.audio_path.replace(".mp3", ".wav")):
            os.remove(self.audio_path.replace(".mp3", ".wav"))

        self.DC.mp3_to_wav(self.audio_path)

        chord_list = ['A','A#','B','C','C#','D','D#','E','F','F#','G','G#','Am',
                      'A#m','Bm','Cm','C#m','Dm','D#m','Em','Fm','F#m','Gm','G#m','N']

        predictions_shape, final_preds = self.DC.predict_seq()

        start = 0
        end = predictions_shape[1]
        time_list = np.round(np.arange(start//10, end//10, 0.1), 1)

        final_preds_chords = [chord_list[int(chord_idx)] for chord_idx in final_preds]
        final_preds_chords = final_preds_chords[0: len(time_list)]

        return self._generate_chord_detection_list(time_list, final_preds_chords)

    def _generate_chord_detection_list(self, time_list, final_preds_chords):
        start = time_list[0]
        chords_list = []
        for i in range(0, len(time_list)):
            if(i == len(time_list) - 2): #penultimo seg
                chords_list.append([start, time_list[i + 1], final_preds_chords[i]])
                break
            elif(final_preds_chords[i] != final_preds_chords[i + 1]):
                    chords_list.append([start, time_list[i], final_preds_chords[i]])
                    start = time_list[i + 1]

        for i in range(1, len(chords_list)):
            chords_list[i][0] = chords_list[i-1][1]

        chords_list = [tuple(seg) for seg in chords_list]
        
        return chords_list

In [12]:
song_path = "Set1/353.mp3"
pred = ChordPrediction(song_path)
chors_list = pred.predict()

print(chors_list)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

1/1 [==============================] - 0s 193ms/step
[(0.0, 1.4, 'N'), (1.4, 2.8, 'F#'), (2.8, 4.6, 'F#m'), (4.6, 5.4, 'B'), (5.4, 6.4, 'A'), (6.4, 9.2, 'F#m'), (9.2, 10.4, 'A'), (10.4, 11.6, 'E'), (11.6, 12.1, 'A'), (12.1, 16.0, 'F#m'), (16.0, 16.7, 'A'), (16.7, 16.9, 'G#m')]
